# Text Attack Tensorflow Exporation

This notebook explores the text attack module and its recipes against a Tensorflow LSTM model. The vectorizer is a padded sequence. 7 attacks were successfully implemented. The chosen problem set is reviews from rotten tomatoes and the classification task is to label positive or negative sentiment.

## Importing

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import datasets
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.corpus import wordnet
from nltk.tokenize import RegexpTokenizer
import nltk
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Embedding, Input, LSTM, SimpleRNN, Conv1D, MaxPool1D, Bidirectional,\
Dropout, MaxPooling1D, Activation
from sklearn import metrics

## Preprocessing

In [2]:
# Using rotten tomatos IMBD dataset for this
dataset = datasets.load_dataset('rotten_tomatoes')

train_df = pd.DataFrame(dataset['train'])
test_df = pd.DataFrame(dataset['test'])

Found cached dataset rotten_tomatoes (C:/Users/mattl/.cache/huggingface/datasets/rotten_tomatoes/default/1.0.0/40d411e45a6ce3484deed7cc15b82a53dad9a72aafd9f86f8f227134bec5ca46)


  0%|          | 0/3 [00:00<?, ?it/s]

In [3]:
#Function for preprocessing
def preprocess(row):

    row = re.sub('[%\n]', ' ', row)
    row = re.sub('[%’""-+\,.:<>-]', '', row)
    row = row.lstrip('[').rstrip(']')
    row = re.sub('[^A-Za-z0-9 ]+','',row)

    #lower case and remove white spaces
    row = row.lower()
    row = row.strip()

    #tokenize and remove stopwords
    tokens = tokenizer.tokenize(row)
    tokens = [w for w in tokens if w not in stop_words]
    lemma = [wnl.lemmatize(w) for w in tokens]
    return lemma

def rejoin_words(row):
    joined_words = ( " ".join(row))
    return joined_words

In [4]:
# Tokenize and remove stopwords
tokenizer = RegexpTokenizer(r'\w+')
# uncomment if need to download
#nltk.download("stopwords") 
#nltk.download('punkt')

stop_words =set(stopwords.words("english"))
wnl = nltk.stem.WordNetLemmatizer()
tokenizer = RegexpTokenizer(r'\w+')

for df in [train_df,test_df]:
    df['words'] = df['text'].apply(preprocess)
    df['processed'] = df['words'].apply(rejoin_words)

In [5]:
# tokenize to sequence, padding
# get list of all words
total_word = []
max_len = 0
for i in train_df["words"].tolist():
    max_len = max(max_len, len(i))
    for j in i:
        total_word.append(j)
total_word = list(set(total_word))

# tokenize to sequences
tokenizer = Tokenizer(num_words = len(total_word))
tokenizer.fit_on_texts(train_df["processed"])
train_sequences = tokenizer.texts_to_sequences(train_df["processed"])
test_sequences = tokenizer.texts_to_sequences(test_df["processed"])

# padding
padded_train = pad_sequences(train_sequences, maxlen = max_len, padding = 'post', truncating = 'post')
padded_test = pad_sequences(test_sequences, maxlen = max_len, padding = 'post', truncating = 'post') 
print("The padded encoding for document\n",train_df["processed"][0],"\n is : ",padded_train[0])

# y-values
y_train = train_df['label'].values
y_test = test_df['label'].values

The padded encoding for document
 rock destined 21st century new conan he going make splash even greater arnold schwarzenegger jeanclaud van damme steven segal 
 is :  [ 480 2493 2933  783   37 5636  212  147    5 2494   10 2171 1193 1719
 8188 1296 5637  784 8189    0    0    0    0    0    0    0    0    0
    0    0    0]


## LSTM model

In [6]:
# Sequential Model
model = Sequential()
model.add(Embedding(len(total_word), output_dim = 180))
model.add(Bidirectional(LSTM(150)))

# Dense layers
model.add(Dropout(0.1))
model.add(Dense(128, activation = 'relu'))
model.add(Dropout(0.1))
model.add(Dense(1,activation= 'sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['binary_accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 180)         2957940   
                                                                 
 bidirectional (Bidirectiona  (None, 300)              397200    
 l)                                                              
                                                                 
 dropout (Dropout)           (None, 300)               0         
                                                                 
 dense (Dense)               (None, 128)               38528     
                                                                 
 dropout_1 (Dropout)         (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 1)                 129       
                                                        

In [7]:
# train the model
model.fit(padded_train, y_train, batch_size = 50, validation_data = (padded_test, y_test), epochs = 2)

Epoch 1/2
171/171 [==============================] - 19s 94ms/step - loss: 0.5898 - binary_accuracy: 0.6662 - val_loss: 0.5147 - val_binary_accuracy: 0.7420
Epoch 2/2
171/171 [==============================] - 25s 148ms/step - loss: 0.2828 - binary_accuracy: 0.8832 - val_loss: 0.5237 - val_binary_accuracy: 0.7542


In [8]:
lstm_pred_tr_pr = [i[0] for i in model.predict(padded_train)]
lstm_pred_te_pr = [i[0] for i in model.predict(padded_test) ]
lstm_pred_tr = [int(i>=0.5) for i in lstm_pred_tr_pr]
lstm_pred_te = [int(i>=0.5) for i in lstm_pred_te_pr]

34/34 [==============================] - 0s 11ms/step


In [9]:
print("train accuracy: ", metrics.accuracy_score(y_train, lstm_pred_tr))
print("test accuracy: ", metrics.accuracy_score(y_test, lstm_pred_te))

train accuracy:  0.9705744431418523
test accuracy:  0.7542213883677298


In [10]:
pred = model.predict(padded_train)

267/267 [==============================] - 2s 9ms/step


In [11]:
pred

array([[0.9916849 ],
       [0.9832622 ],
       [0.9604677 ],
       ...,
       [0.33998555],
       [0.017466  ],
       [0.04674144]], dtype=float32)

In [12]:
# Create a new array with expanded shape
new_pred = np.empty((pred.shape[0], 2), dtype=np.float32)

# Copy values to the new array
for i in range(pred.shape[0]):
    value = pred[i][0]
    new_pred[i] = [value, 1 - value]

In [13]:
new_pred

array([[0.9916849 , 0.00831509],
       [0.9832622 , 0.01673782],
       [0.9604677 , 0.0395323 ],
       ...,
       [0.33998555, 0.66001445],
       [0.017466  , 0.982534  ],
       [0.04674144, 0.9532586 ]], dtype=float32)

In [14]:
pred[3]

array([0.29616064], dtype=float32)

# Text Attack Attacks

In [15]:
import numpy as np
import torch

from textattack.models.wrappers import ModelWrapper
from textattack.datasets import HuggingFaceDataset
from textattack.attack_recipes import TextFoolerJin2019
from textattack import Attacker

class CustomTensorFlowModelWrapper(ModelWrapper):
    def __init__(self, model,total_word):
        self.model = model
        self.total_word = total_word

        
    # How this works is TextAttack needs to know how the model preprocesses the text
    # Given that, you can then use the model to predict.
    def __call__(self, text_input_list):
        tokenizer = Tokenizer(num_words = len(self.total_word))
        tokenizer.fit_on_texts(text_input_list)
        sequences = tokenizer.texts_to_sequences(text_input_list)

        # padding
        padded = pad_sequences(sequences, maxlen = max_len, padding = 'post', truncating = 'post')
        pred = model.predict(padded)
        
        # Prediction array must have a positive and negative value for textattack
        new_pred = np.empty((pred.shape[0], 2), dtype=np.float32)

        for i in range(pred.shape[0]):
            value = pred[i][0]
            new_pred[i] = [value, 1 - value]
    
        return new_pred

In [16]:
CustomTensorFlowModelWrapper(model=model, total_word=total_word)

In [17]:
def attack_tensor(attack_recipe,model_wrapper,text_data):
    attack = attack_recipe.build(model_wrapper)
    
    attacker = Attacker(attack,text_data)
    attacker.attack_dataset()

In [18]:
model_wrapper = CustomTensorFlowModelWrapper(model,total_word)

text_data = HuggingFaceDataset("rotten_tomatoes", None, "train")

Found cached dataset rotten_tomatoes (C:/Users/mattl/.cache/huggingface/datasets/rotten_tomatoes/default/1.0.0/40d411e45a6ce3484deed7cc15b82a53dad9a72aafd9f86f8f227134bec5ca46)


  0%|          | 0/3 [00:00<?, ?it/s]

textattack: Loading datasets dataset rotten_tomatoes, split train.


## TextFoolerJin2019

This attack recipe is a word based subsistution attack. It does so by subsituting words with a similar meaning. 

In [19]:
from textattack.attack_recipes import TextFoolerJin2019
attack_tensor(TextFoolerJin2019,model_wrapper,text_data)

textattack: Unknown if model of class <class 'keras.engine.sequential.Sequential'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.840845057
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']
       

  0%|                                                                                           | 0/10 [00:00<?, ?it/s]

1/1 [==============================] - 0s 26ms/step


[Succeeded / Failed / Skipped / Total] 0 / 0 / 1 / 1:  10%|██▉                          | 1/10 [00:00<00:01,  8.20it/s]

...

--------------------------------------------- Result 8 ---------------------------------------------
[[Negative (82%)]] --> [[[SKIPPED]]]

perhaps no picture ever made has more literally showed that the road to hell is paved with good intentions .


1/1 [==============================] - 0s 19ms/step


[Succeeded / Failed / Skipped / Total] 2 / 0 / 7 / 9:  90%|██████████████████████████   | 9/10 [00:08<00:00,  1.07it/s]

--------------------------------------------- Result 9 ---------------------------------------------
[[Negative (82%)]] --> [[[SKIPPED]]]

steers turns in a snappy screenplay that curls at the edges ; it's so clever you want to hate it . but he somehow pulls it off .


1/1 [==============================] - 0s 24ms/step


[Succeeded / Failed / Skipped / Total] 3 / 0 / 7 / 10: 100%|███████████████████████████| 10/10 [00:08<00:00,  1.15it/s]

--------------------------------------------- Result 10 ---------------------------------------------
[[Positive (51%)]] --> [[Negative (67%)]]

take care of my cat offers a [[refreshingly]] different slice of asian cinema .

take care of my cat offers a [[brilliantly]] different slice of asian cinema .



+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 3      |
| Number of failed attacks:     | 0      |
| Number of skipped attacks:    | 7      |
| Original accuracy:            | 30.0%  |
| Accuracy under attack:        | 0.0%   |
| Attack success rate:          | 100.0% |
| Average perturbed word %:     | 15.06% |
| Average num. words per input: | 19.5   |
| Avg num queries:              | 38.0   |
+-------------------------------+--------+


It seems like the pad sequences of the tensorflow model is more robust at fending off this algorithim's attack than the bag of words and tf-idf of the sklearn models (see: Text Attack SKlearn Exploration)

## A2T_yoo_2021 (N/A)

The above attack is meant to be used against BERT

## Checklist Ribeiro 2020

As per the documentation, this attack focuses on a number of attacks used in the Invariance Testing Method: Contraction, Extension, Changing Names, Number, Location.

In [21]:
from textattack.attack_recipes import CheckList2020
attack_tensor(CheckList2020,model_wrapper,text_data)

textattack: Unknown if model of class <class 'keras.engine.sequential.Sequential'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.


Attack(
  (search_method): GreedySearch
  (goal_function):  UntargetedClassification
  (transformation):  CompositeTransformation(
    (0): WordSwapExtend
    (1): WordSwapContract
    (2): WordSwapChangeName
    (3): WordSwapChangeNumber
    (4): WordSwapChangeLocation
    )
  (constraints): 
    (0): RepeatModification
  (is_black_box):  True
) 



  0%|                                                                                           | 0/10 [00:00<?, ?it/s]

1/1 [==============================] - 0s 21ms/step


[Succeeded / Failed / Skipped / Total] 0 / 0 / 1 / 1:  10%|██▉                          | 1/10 [00:00<00:00, 15.15it/s]

...

[Succeeded / Failed / Skipped / Total] 0 / 2 / 7 / 9:  90%|██████████████████████████   | 9/10 [00:03<00:00,  2.36it/s]

--------------------------------------------- Result 9 ---------------------------------------------
[[Negative (82%)]] --> [[[SKIPPED]]]

steers turns in a snappy screenplay that curls at the edges ; it's so clever you want to hate it . but he somehow pulls it off .


1/1 [==============================] - 0s 20ms/step


[Succeeded / Failed / Skipped / Total] 0 / 3 / 7 / 10: 100%|███████████████████████████| 10/10 [00:04<00:00,  2.35it/s]

--------------------------------------------- Result 10 ---------------------------------------------
[[Positive (51%)]] --> [[[FAILED]]]

take care of my cat offers a refreshingly different slice of asian cinema .



+-------------------------------+-------+
| Attack Results                |       |
+-------------------------------+-------+
| Number of successful attacks: | 0     |
| Number of failed attacks:     | 3     |
| Number of skipped attacks:    | 7     |
| Original accuracy:            | 30.0% |
| Accuracy under attack:        | 30.0% |
| Attack success rate:          | 0.0%  |
| Average perturbed word %:     | nan%  |
| Average num. words per input: | 19.5  |
| Avg num queries:              | 1.0   |
+-------------------------------+-------+



C:\Users\mattl\anaconda3\lib\site-packages\textattack\metrics\attack_metrics\words_perturbed.py:83: RuntimeWarning: Mean of empty slice.
  average_perc_words_perturbed = self.perturbed_word_percentages.mean()
C:\Users\mattl\anaconda3\lib\site-packages\numpy\core\_methods.py:192: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


Compared to Sklearn, again we see that the pad sequences is more robust in defending against such an attack which all the attacks being fialures.

## Clare Li 2020 (Not Working)

This algorithim constructs adversarial examples to test the robustness of NLP Models

In [22]:
from textattack.attack_recipes import CLARE2020
attack_tensor(CLARE2020,model_wrapper,text_data)

If you want to use `RobertaLMHeadModel` as a standalone, add `is_decoder=True.`
textattack: Unknown if model of class <class 'keras.engine.sequential.Sequential'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.


Attack(
  (search_method): GreedySearch
  (goal_function):  UntargetedClassification
  (transformation):  CompositeTransformation(
    (0): WordSwapMaskedLM(
        (method):  bae
        (masked_lm_name):  RobertaForCausalLM
        (max_length):  512
        (max_candidates):  50
        (min_confidence):  0.0005
      )
    (1): WordInsertionMaskedLM(
        (masked_lm_name):  RobertaForCausalLM
        (max_length):  512
        (max_candidates):  50
        (min_confidence):  0.0
      )
    (2): WordMergeMaskedLM(
        (masked_lm_name):  RobertaForCausalLM
        (max_length):  512
        (max_candidates):  50
        (min_confidence):  0.005
      )
    )
  (constraints): 
    (0): UniversalSentenceEncoder(
        (metric):  cosine
        (threshold):  0.7
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  True
      )
    (1): RepeatModification
    (2): StopwordModification
  (is_black_box):  True
) 



  0%|                                                                                           | 0/10 [00:00<?, ?it/s]

1/1 [==============================] - 0s 18ms/step


[Succeeded / Failed / Skipped / Total] 0 / 0 / 1 / 1:  10%|██▉                          | 1/10 [00:00<00:00, 19.61it/s]

--------------------------------------------- Result 1 ---------------------------------------------
[[Negative (74%)]] --> [[[SKIPPED]]]

the rock is destined to be the 21st century's new " conan " and that he's going to make a splash even greater than arnold schwarzenegger , jean-claud van damme or steven segal .


1/1 [==============================] - 0s 16ms/step


[Succeeded / Failed / Skipped / Total] 0 / 0 / 2 / 2:  20%|█████▊                       | 2/10 [00:00<00:00, 20.83it/s]

--------------------------------------------- Result 2 ---------------------------------------------
[[Negative (67%)]] --> [[[SKIPPED]]]

the gorgeously elaborate continuation of " the lord of the rings " trilogy is so huge that a column of words cannot adequately describe co-writer/director peter jackson's expanded vision of j . r . r . tolkien's middle-earth .


1/1 [==============================] - 0s 17ms/step


KeyError: 'upos'

## Faster Alzantot Genetic Algorithm (Not Working) 

This Algorithm implements an optimization algorithm inspired by the proccess of natural selection.

In [23]:
from textattack.attack_recipes import FasterGeneticAlgorithmJia2019
attack_tensor(FasterGeneticAlgorithmJia2019,model_wrapper,text_data)

C:\Users\mattl\anaconda3\lib\site-packages\torch\nn\modules\rnn.py:82: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.01 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "
textattack: Unknown if model of class <class 'keras.engine.sequential.Sequential'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.


Attack(
  (search_method): AlzantotGeneticAlgorithm(
    (pop_size):  60
    (max_iters):  40
    (temp):  0.3
    (give_up_if_no_improvement):  False
    (post_crossover_check):  False
    (max_crossover_retries):  20
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  8
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): MaxWordsPerturbed(
        (max_percent):  0.2
        (compare_against_original):  True
      )
    (1): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (max_mse_dist):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (2): LearningToWriteLanguageModel(
        (max_log_prob_diff):  5.0
        (compare_against_original):  True
      )
    (3): RepeatModification
    (4): StopwordModification
  (is_black_box):  True
) 




  0%|                                                                                           | 0/10 [00:00<?, ?it/s]

1/1 [==============================] - 0s 21ms/step



[Succeeded / Failed / Skipped / Total] 0 / 0 / 1 / 1:  10%|██▉                          | 1/10 [00:00<00:00, 14.71it/s]

--------------------------------------------- Result 1 ---------------------------------------------
[[Negative (74%)]] --> [[[SKIPPED]]]

the rock is destined to be the 21st century's new " conan " and that he's going to make a splash even greater than arnold schwarzenegger , jean-claud van damme or steven segal .


1/1 [==============================] - 0s 21ms/step



[Succeeded / Failed / Skipped / Total] 0 / 0 / 1 / 1:  20%|█████▊                       | 2/10 [00:00<00:00, 15.50it/s]
[Succeeded / Failed / Skipped / Total] 0 / 0 / 2 / 2:  20%|█████▊                       | 2/10 [00:00<00:00, 15.38it/s]

--------------------------------------------- Result 2 ---------------------------------------------
[[Negative (67%)]] --> [[[SKIPPED]]]

the gorgeously elaborate continuation of " the lord of the rings " trilogy is so huge that a column of words cannot adequately describe co-writer/director peter jackson's expanded vision of j . r . r . tolkien's middle-earth .


1/1 [==============================] - 0s 19ms/step


IndexError: index is out of bounds for dimension with size 0


[Succeeded / Failed / Skipped / Total] 0 / 0 / 2 / 2:  20%|█████▊                       | 2/10 [00:14<00:58,  7.36s/it]

## Hotflip (hotflip_ebrahimi_2017) - Not Working

In [26]:
from textattack.attack_recipes import HotFlipEbrahimi2017
attack_tensor(HotFlipEbrahimi2017,model_wrapper,text_data)

AttributeError: 'CustomTensorFlowModelWrapper' object has no attribute 'tokenizer'

## Improved Genetic Algorithm (Iga Wang 2019)

This is an improved genetic algorithim. It also poisons the input to cause training accuracy deficiency.

In [25]:
from textattack.attack_recipes import IGAWang2019
attack_tensor(IGAWang2019,model_wrapper,text_data)

textattack: Unknown if model of class <class 'keras.engine.sequential.Sequential'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.


Attack(
  (search_method): ImprovedGeneticAlgorithm(
    (pop_size):  60
    (max_iters):  20
    (temp):  0.3
    (give_up_if_no_improvement):  False
    (post_crossover_check):  False
    (max_crossover_retries):  20
    (max_replace_times_per_index):  5
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): MaxWordsPerturbed(
        (max_percent):  0.2
        (compare_against_original):  True
      )
    (1): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (max_mse_dist):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  False
      )
    (2): StopwordModification
  (is_black_box):  True
) 






  0%|                                                                                           | 0/10 [00:00<?, ?it/s]

1/1 [==============================] - 0s 20ms/step





[Succeeded / Failed / Skipped / Total] 0 / 0 / 1 / 1:  10%|██▉                          | 1/10 [00:00<00:00, 16.95it/s]

...

--------------------------------------------- Result 8 ---------------------------------------------
[[Negative (82%)]] --> [[[SKIPPED]]]

perhaps no picture ever made has more literally showed that the road to hell is paved with good intentions .


1/1 [==============================] - 0s 21ms/step





[Succeeded / Failed / Skipped / Total] 2 / 0 / 7 / 9:  90%|██████████████████████████   | 9/10 [00:00<00:00,  9.39it/s]

--------------------------------------------- Result 9 ---------------------------------------------
[[Negative (82%)]] --> [[[SKIPPED]]]

steers turns in a snappy screenplay that curls at the edges ; it's so clever you want to hate it . but he somehow pulls it off .


1/1 [==============================] - 0s 27ms/step





[Succeeded / Failed / Skipped / Total] 2 / 0 / 7 / 9: 100%|████████████████████████████| 10/10 [00:01<00:00,  6.38it/s]


[Succeeded / Failed / Skipped / Total] 3 / 0 / 7 / 10: 100%|███████████████████████████| 10/10 [00:01<00:00,  6.30it/s]

--------------------------------------------- Result 10 ---------------------------------------------
[[Positive (51%)]] --> [[Negative (70%)]]

take care of my cat [[offers]] a refreshingly different slice of asian cinema .

take care of my cat [[offering]] a refreshingly different slice of asian cinema .



+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 3      |
| Number of failed attacks:     | 0      |
| Number of skipped attacks:    | 7      |
| Original accuracy:            | 30.0%  |
| Accuracy under attack:        | 0.0%   |
| Attack success rate:          | 100.0% |
| Average perturbed word %:     | 15.06% |
| Average num. words per input: | 19.5   |
| Avg num queries:              | 52.67  |
+-------------------------------+--------+


## Input Reduction

This attack removes words in the input. 

In [27]:
from textattack.attack_recipes import InputReductionFeng2018
attack_tensor(InputReductionFeng2018,model_wrapper,text_data)

textattack: Unknown if model of class <class 'keras.engine.sequential.Sequential'> compatible with goal function <class 'textattack.goal_functions.classification.input_reduction.InputReduction'>.


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  InputReduction(
    (maximizable):  True
  )
  (transformation):  WordDeletion
  (constraints): 
    (0): RepeatModification
    (1): StopwordModification
  (is_black_box):  True
) 






  0%|                                                                                           | 0/10 [00:00<?, ?it/s]

1/1 [==============================] - 0s 22ms/step





[Succeeded / Failed / Skipped / Total] 0 / 0 / 1 / 1:  10%|██▉                          | 1/10 [00:00<00:00, 15.15it/s]

...

--------------------------------------------- Result 8 ---------------------------------------------
[[Negative (82%)]] --> [[[SKIPPED]]]

perhaps no picture ever made has more literally showed that the road to hell is paved with good intentions .


1/1 [==============================] - 0s 20ms/step





[Succeeded / Failed / Skipped / Total] 2 / 0 / 7 / 9:  90%|██████████████████████████   | 9/10 [00:01<00:00,  7.65it/s]

--------------------------------------------- Result 9 ---------------------------------------------
[[Negative (82%)]] --> [[[SKIPPED]]]

steers turns in a snappy screenplay that curls at the edges ; it's so clever you want to hate it . but he somehow pulls it off .


1/1 [==============================] - 0s 19ms/step





[Succeeded / Failed / Skipped / Total] 2 / 0 / 7 / 9: 100%|████████████████████████████| 10/10 [00:01<00:00,  5.59it/s]


[Succeeded / Failed / Skipped / Total] 3 / 0 / 7 / 10: 100%|███████████████████████████| 10/10 [00:01<00:00,  5.55it/s]

--------------------------------------------- Result 10 ---------------------------------------------
[[Positive (51%)]] --> [[Positive (66%)]]

take care [[of]] my [[cat]] [[offers]] a [[refreshingly]] different slice of [[asian]] cinema .

take care my a different slice of cinema .



+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 3      |
| Number of failed attacks:     | 0      |
| Number of skipped attacks:    | 7      |
| Original accuracy:            | 30.0%  |
| Accuracy under attack:        | 0.0%   |
| Attack success rate:          | 100.0% |
| Average perturbed word %:     | 40.38% |
| Average num. words per input: | 19.5   |
| Avg num queries:              | 15.33  |
+-------------------------------+--------+


## Kuleshov 2017

This is an algorithim designed to generate adversial examples to poison the dataset.

In [28]:
from textattack.attack_recipes import Kuleshov2017
attack_tensor(Kuleshov2017,model_wrapper,text_data)

textattack: Unknown if model of class <class 'keras.engine.sequential.Sequential'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.


Attack(
  (search_method): GreedySearch
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  15
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): MaxWordsPerturbed(
        (max_percent):  0.5
        (compare_against_original):  True
      )
    (1): ThoughtVector(
        (word_embedding):  WordEmbedding
        (metric):  max_euclidean
        (threshold):  -0.2
        (window_size):  inf
        (skip_text_shorter_than_window):  False
        (compare_against_original):  True
      )
    (2): GPT2(
        (max_log_prob_diff):  2.0
        (compare_against_original):  True
      )
    (3): RepeatModification
    (4): StopwordModification
  (is_black_box):  True
) 






  0%|                                                                                           | 0/10 [00:00<?, ?it/s]

1/1 [==============================] - 0s 20ms/step





[Succeeded / Failed / Skipped / Total] 0 / 0 / 1 / 1:  10%|██▉                          | 1/10 [00:00<00:00, 16.67it/s]

...




[Succeeded / Failed / Skipped / Total] 0 / 0 / 9 / 9:  90%|██████████████████████████   | 9/10 [00:00<00:00, 16.67it/s]

--------------------------------------------- Result 9 ---------------------------------------------
[[Negative (82%)]] --> [[[SKIPPED]]]

steers turns in a snappy screenplay that curls at the edges ; it's so clever you want to hate it . but he somehow pulls it off .


1/1 [==============================] - 0s 19ms/step





[Succeeded / Failed / Skipped / Total] 0 / 0 / 9 / 9: 100%|████████████████████████████| 10/10 [00:00<00:00, 16.75it/s]


[Succeeded / Failed / Skipped / Total] 0 / 0 / 10 / 10: 100%|██████████████████████████| 10/10 [00:00<00:00, 16.64it/s]

--------------------------------------------- Result 10 ---------------------------------------------
[[Positive (51%)]] --> [[[SKIPPED]]]

take care of my cat offers a refreshingly different slice of asian cinema .



+-------------------------------+------+
| Attack Results                |      |
+-------------------------------+------+
| Number of successful attacks: | 0    |
| Number of failed attacks:     | 0    |
| Number of skipped attacks:    | 10   |
| Original accuracy:            | 0.0% |
| Accuracy under attack:        | 0.0% |
| Attack success rate:          | 0%   |
| Average perturbed word %:     | nan% |
| Average num. words per input: | 19.5 |
| Avg num queries:              | nan  |
+-------------------------------+------+



C:\Users\mattl\anaconda3\lib\site-packages\textattack\metrics\attack_metrics\attack_queries.py:39: RuntimeWarning: Mean of empty slice.
  avg_num_queries = self.num_queries.mean()


The attacks appear to not work for this model or rather all of them were skipped.

## Morpheus Tan 2020 (doesn't work)

Seems to be for BERT

In [29]:
from textattack.attack_recipes import MorpheusTan2020
attack_tensor(MorpheusTan2020,model_wrapper,text_data)

textattack: Unknown if model of class <class 'keras.engine.sequential.Sequential'> compatible with goal function <class 'textattack.goal_functions.text.minimize_bleu.MinimizeBleu'>.


Attack(
  (search_method): GreedySearch
  (goal_function):  MinimizeBleu(
    (maximizable):  False
    (target_bleu):  0.0
  )
  (transformation):  WordSwapInflections
  (constraints): 
    (0): RepeatModification
    (1): StopwordModification
  (is_black_box):  True
) 






  0%|                                                                                           | 0/10 [00:00<?, ?it/s]

1/1 [==============================] - 0s 22ms/step


TypeError: Invalid text_input type <class 'torch.Tensor'> (required str or OrderedDict)

## Pruthi 2019

This is an algorithim that swaps characters either by inserting or deleting them. It targets characters that are on adjacent keys on a QWERTY keyboard. 

In [30]:
from textattack.attack_recipes import Pruthi2019
attack_tensor(Pruthi2019,model_wrapper,text_data)

textattack: Unknown if model of class <class 'keras.engine.sequential.Sequential'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.


Attack(
  (search_method): GreedySearch
  (goal_function):  UntargetedClassification
  (transformation):  CompositeTransformation(
    (0): WordSwapNeighboringCharacterSwap(
        (random_one):  False
      )
    (1): WordSwapRandomCharacterDeletion(
        (random_one):  False
      )
    (2): WordSwapRandomCharacterInsertion(
        (random_one):  False
      )
    (3): WordSwapQWERTY
    )
  (constraints): 
    (0): MaxWordsPerturbed(
        (max_num_words):  1
        (compare_against_original):  True
      )
    (1): MinWordLength
    (2): StopwordModification
    (3): RepeatModification
  (is_black_box):  True
) 







  0%|                                                                                           | 0/10 [00:00<?, ?it/s]

1/1 [==============================] - 0s 21ms/step






[Succeeded / Failed / Skipped / Total] 0 / 0 / 1 / 1:  10%|██▉                          | 1/10 [00:00<00:00, 15.87it/s]

...

--------------------------------------------- Result 8 ---------------------------------------------
[[Negative (82%)]] --> [[[SKIPPED]]]

perhaps no picture ever made has more literally showed that the road to hell is paved with good intentions .


1/1 [==============================] - 0s 18ms/step






[Succeeded / Failed / Skipped / Total] 2 / 0 / 7 / 9:  90%|██████████████████████████   | 9/10 [00:01<00:00,  7.24it/s]

--------------------------------------------- Result 9 ---------------------------------------------
[[Negative (82%)]] --> [[[SKIPPED]]]

steers turns in a snappy screenplay that curls at the edges ; it's so clever you want to hate it . but he somehow pulls it off .


1/1 [==============================] - 0s 20ms/step






[Succeeded / Failed / Skipped / Total] 2 / 0 / 7 / 9: 100%|████████████████████████████| 10/10 [00:01<00:00,  5.64it/s]



[Succeeded / Failed / Skipped / Total] 3 / 0 / 7 / 10: 100%|███████████████████████████| 10/10 [00:01<00:00,  5.61it/s]

--------------------------------------------- Result 10 ---------------------------------------------
[[Positive (51%)]] --> [[Negative (86%)]]

take [[care]] of my cat offers a refreshingly different slice of asian cinema .

take [[caTre]] of my cat offers a refreshingly different slice of asian cinema .



+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 3      |
| Number of failed attacks:     | 0      |
| Number of skipped attacks:    | 7      |
| Original accuracy:            | 30.0%  |
| Accuracy under attack:        | 0.0%   |
| Attack success rate:          | 100.0% |
| Average perturbed word %:     | 15.06% |
| Average num. words per input: | 19.5   |
| Avg num queries:              | 219.0  |
+-------------------------------+--------+


## Pso Zang 2020 (doesn't work)

In [31]:
from textattack.attack_recipes import PSOZang2020
attack_tensor(PSOZang2020,model_wrapper,text_data)

textattack: Downloading https://textattack.s3.amazonaws.com/transformations/hownet/word_candidates_sense.pkl.




  0%|                                                                                      | 0.00/8.39M [00:00<?, ?B/s]



  1%|▋                                                                             | 68.6k/8.39M [00:00<00:19, 430kB/s]



  4%|██▊                                                                            | 295k/8.39M [00:00<00:08, 975kB/s]



  8%|██████▍                                                                       | 695k/8.39M [00:00<00:03, 1.98MB/s]



 16%|████████████▏                                                                | 1.32M/8.39M [00:00<00:02, 3.36MB/s]



 32%|████████████████████████▋                                                    | 2.69M/8.39M [00:00<00:00, 6.60MB/s]



 48%|█████████████████████████████████████                                        | 4.04M/8.39M [00:00<00:00, 8.73MB/s]



 63%|████████████████████

Attack(
  (search_method): ParticleSwarmOptimization(
    (pop_size):  60
    (max_iters):  20
    (post_turn_check):  True
    (max_turn_retries):  20
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapHowNet(
    (max_candidates):  -1
  )
  (constraints): 
    (0): RepeatModification
    (1): StopwordModification
    (2): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']
        (columns_to_ignore):  {'premise'}
      )
  (is_black_box):  True
) 



  0%|                                                                                           | 0/10 [00:00<?, ?it/s]

1/1 [==============================] - 0s 20ms/step


[Succeeded / Failed / Skipped / Total] 0 / 0 / 1 / 1:  10%|██▉                          | 1/10 [00:00<00:00, 17.54it/s]

...

[Succeeded / Failed / Skipped / Total] 0 / 0 / 2 / 2:  20%|█████▊                       | 2/10 [00:00<00:00, 17.86it/s]

--------------------------------------------- Result 2 ---------------------------------------------
[[Negative (67%)]] --> [[[SKIPPED]]]

the gorgeously elaborate continuation of " the lord of the rings " trilogy is so huge that a column of words cannot adequately describe co-writer/director peter jackson's expanded vision of j . r . r . tolkien's middle-earth .


1/1 [==============================] - 0s 19ms/step


  0%|                                                                                           | 0/10 [12:58<?, ?it/s]


KeyError: 'upos'

[Succeeded / Failed / Skipped / Total] 0 / 0 / 2 / 2:  20%|█████▊                       | 2/10 [00:16<01:06,  8.32s/it]

## PWWS Ren 2019

This algorithim is another word swap algorithim that goes for synonyms.

In [32]:
from textattack.attack_recipes import PWWSRen2019
attack_tensor(PWWSRen2019,model_wrapper,text_data)

[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\mattl\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
textattack: Unknown if model of class <class 'keras.engine.sequential.Sequential'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  weighted-saliency
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapWordNet
  (constraints): 
    (0): RepeatModification
    (1): StopwordModification
  (is_black_box):  True
) 




  0%|                                                                                           | 0/10 [00:00<?, ?it/s]

1/1 [==============================] - 0s 21ms/step



[Succeeded / Failed / Skipped / Total] 0 / 0 / 1 / 1:  10%|██▉                          | 1/10 [00:00<00:00, 16.67it/s]

...

--------------------------------------------- Result 9 ---------------------------------------------
[[Negative (82%)]] --> [[[SKIPPED]]]

steers turns in a snappy screenplay that curls at the edges ; it's so clever you want to hate it . but he somehow pulls it off .


1/1 [==============================] - 0s 23ms/step



[Succeeded / Failed / Skipped / Total] 2 / 0 / 7 / 9: 100%|████████████████████████████| 10/10 [00:01<00:00,  5.82it/s]
[Succeeded / Failed / Skipped / Total] 3 / 0 / 7 / 10: 100%|███████████████████████████| 10/10 [00:01<00:00,  5.78it/s]

--------------------------------------------- Result 10 ---------------------------------------------
[[Positive (51%)]] --> [[Negative (70%)]]

take care of my cat [[offers]] a refreshingly different slice of asian cinema .

take care of my cat [[offering]] a refreshingly different slice of asian cinema .




[Succeeded / Failed / Skipped / Total] 3 / 0 / 7 / 10: 100%|███████████████████████████| 10/10 [00:01<00:00,  5.77it/s]


+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 3      |
| Number of failed attacks:     | 0      |
| Number of skipped attacks:    | 7      |
| Original accuracy:            | 30.0%  |
| Accuracy under attack:        | 0.0%   |
| Attack success rate:          | 100.0% |
| Average perturbed word %:     | 15.06% |
| Average num. words per input: | 19.5   |
| Avg num queries:              | 72.0   |
+-------------------------------+--------+


## Seq2sick Cheng 2018 Blackbox (doesn't work)

In [33]:
from textattack.attack_recipes import Seq2SickCheng2018BlackBox
attack_tensor(Seq2SickCheng2018BlackBox,model_wrapper,text_data)

textattack: Unknown if model of class <class 'keras.engine.sequential.Sequential'> compatible with goal function <class 'textattack.goal_functions.text.non_overlapping_output.NonOverlappingOutput'>.


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  unk
  )
  (goal_function):  NonOverlappingOutput
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): LevenshteinEditDistance(
        (max_edit_distance):  30
        (compare_against_original):  True
      )
    (1): RepeatModification
    (2): StopwordModification
  (is_black_box):  True
) 




  0%|                                                                                           | 0/10 [00:00<?, ?it/s]

1/1 [==============================] - 0s 20ms/step


TypeError: split() missing 1 required positional argument: 'split_size'

## Text Bugger Li 2019

This is a rather malicious algorithim that causes failures in deep learning algorithims. What makes is particularly malicious is the replacement looks identical to human eyes so it is incredibly difficult to tell when the data has been poisoned.

In [34]:
from textattack.attack_recipes import TextBuggerLi2018
attack_tensor(TextBuggerLi2018,model_wrapper,text_data)

textattack: Unknown if model of class <class 'keras.engine.sequential.Sequential'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  CompositeTransformation(
    (0): WordSwapRandomCharacterInsertion(
        (random_one):  True
      )
    (1): WordSwapRandomCharacterDeletion(
        (random_one):  True
      )
    (2): WordSwapNeighboringCharacterSwap(
        (random_one):  True
      )
    (3): WordSwapHomoglyphSwap
    (4): WordSwapEmbedding(
        (max_candidates):  5
        (embedding):  WordEmbedding
      )
    )
  (constraints): 
    (0): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8
        (window_size):  inf
        (skip_text_shorter_than_window):  False
        (compare_against_original):  True
      )
    (1): RepeatModification
    (2): StopwordModification
  (is_black_box):  True
) 





  0%|                                                                                           | 0/10 [00:00<?, ?it/s]

1/1 [==============================] - 0s 22ms/step




[Succeeded / Failed / Skipped / Total] 0 / 0 / 1 / 1:  10%|██▉                          | 1/10 [00:00<00:00, 16.13it/s]

...

--------------------------------------------- Result 8 ---------------------------------------------
[[Negative (82%)]] --> [[[SKIPPED]]]

perhaps no picture ever made has more literally showed that the road to hell is paved with good intentions .


1/1 [==============================] - 0s 17ms/step




[Succeeded / Failed / Skipped / Total] 2 / 0 / 7 / 9:  90%|██████████████████████████   | 9/10 [00:06<00:00,  1.44it/s]

--------------------------------------------- Result 9 ---------------------------------------------
[[Negative (82%)]] --> [[[SKIPPED]]]

steers turns in a snappy screenplay that curls at the edges ; it's so clever you want to hate it . but he somehow pulls it off .


1/1 [==============================] - 0s 21ms/step




[Succeeded / Failed / Skipped / Total] 2 / 0 / 7 / 9: 100%|████████████████████████████| 10/10 [00:06<00:00,  1.56it/s]

[Succeeded / Failed / Skipped / Total] 3 / 0 / 7 / 10: 100%|███████████████████████████| 10/10 [00:06<00:00,  1.56it/s]

--------------------------------------------- Result 10 ---------------------------------------------
[[Positive (51%)]] --> [[Negative (58%)]]

take care of my cat offers a [[refreshingly]] different slice of asian cinema .

take care of my cat offers a [[refreshiոgly]] different slice of asian cinema .



+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 3      |
| Number of failed attacks:     | 0      |
| Number of skipped attacks:    | 7      |
| Original accuracy:            | 30.0%  |
| Accuracy under attack:        | 0.0%   |
| Attack success rate:          | 100.0% |
| Average perturbed word %:     | 15.06% |
| Average num. words per input: | 19.5   |
| Avg num queries:              | 22.67  |
+-------------------------------+--------+


# Summary

In addition to the attacks that worked with the Sklearn models, there are more attacks that were successfully implemented against a TensorFlow LSTM model. Notably while the attacks could not attack all the data, the attacks that were successful generally all managed to successfully turn the classification into a false positive or false negative. Compared to sklearn, Tensorflow seems to be more robust but has a particularly nasty algorithim that can be used against it. In order of most effective to least effective the models are the following:
1. Text Bugger Li 2018 (Insidious poisoning, 3 success, 7 skipped)
2. TextFoolerJin2019 = Iga Wang 2019 = Input Reduction = Pruthi2019 (3 success, 7 skipped)
3. Checklist 2020 (3 fail, 7 skipped)
4. Kuleshov 2017 (all skipped)